In [39]:
from public_data.models import OcsgeDiff, CouvertureSol, Departement, Commune
from utils.db import cast_sum
from django.contrib.gis.db.models.functions import Centroid
from django.contrib.gis.db.models import Union
from django.db.models import Q

dept = Departement.objects.filter(name="Gers").first()

In [40]:
qs = (
    OcsgeDiff.objects.filter(Q(is_new_artif=True) | Q(is_new_natural=True))
    .filter(Q(new_matrix__couverture__code_prefix="CS1.1.2.2") | Q(old_matrix__couverture__code_prefix="CS1.1.2.2"))
    .annotate(center=Centroid("mpoly"))
)
qs.count()

6

In [43]:
for item in qs:
    comm = Commune.objects.filter(mpoly__contains=item.mpoly).first()
    print(comm.name, comm.departement)
    print(item.old_matrix, item.new_matrix)

Biganos Gironde
CS2.1.1.1-US5: CS1.1.2.2-US6.1:a
Biganos Gironde
CS2.1.1.1-US5: CS1.1.2.2-US6.1:a
Maîche Doubs
CS2.1.1.1-US1.2: CS1.1.2.2-US2:a
La Roche-en-Brenil Côte-d'Or
CS2.2.1.1-US1.1.3: CS1.1.2.2-US2:a
Courlaoux Jura
CS2.1.1.1-US4.3: CS1.1.2.2-US4.3:a
Saint-Cyr-les-Colons Yonne
CS2.2.1-US1.1: CS1.1.2.2-US4.3:a


In [27]:
map_center = OcsgeDiff.objects.aggregate(center=Centroid(Union("mpoly")))
map_center

{'center': <Point object at 0x7face2378590>}

In [32]:
from ipyleaflet import Map, Marker, GeoJSON, Polygon as iPolygon, Icon, WKTLayer
from public_data.api.serializers import RegionSerializer
from public_data.models import Region
from django.contrib.gis.geos import Polygon as dPolygon

m = Map(center=(map_center["center"].y, map_center["center"].x), zoom=5)
for item in qs:
    center = (item.center.y, item.center.x)
    print(center)
    m.add_layer(Marker(location=center, draggable=False))
m

(47.245559445309034, 6.7823342017161945)
(47.38981669806705, 4.156134189086796)
(44.662606023867085, -1.0003281522399863)
(44.662606023867085, -1.0003281522399863)
(46.68148678493917, 5.431466614590644)
(47.71077127340695, 3.79501600538416)


Map(center=[47.11085844623028, 4.860404177965002], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [38]:
qs = (
    OcsgeDiff.objects.intersect(dept.mpoly)
    .filter(Q(is_new_artif=True) | Q(is_new_natural=True))
    .annotate(
        code_prefix=Case(
            When(
                is_new_artif=True,
                then=F("new_matrix__couverture__code_prefix")
            ),
            default=F("old_matrix__couverture__code_prefix"),
        ),
        label=Case(
            When(is_new_artif=True, then=F("new_matrix__couverture__label")),
            default=F("old_matrix__couverture__label"),
        ),
        label_short=Case(
            When(is_new_artif=True, then=F("new_matrix__couverture__label_short")),
            default=F("old_matrix__couverture__label_short"),
        ),
    )
    .values("code_prefix", "label", "label_short")
    .annotate(
        artif=cast_sum("intersection_area", filter=Q(is_new_artif=True)),
        renat=cast_sum("intersection_area", filter=Q(is_new_natural=True)),
    )
    .order_by("code_prefix", "label", "label_short")
)
qs.count()

4